In [1]:
from keras.models import load_model
import numpy as np
import json

2024-11-21 19:45:12.067725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 19:45:12.095413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 19:45:12.095464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 19:45:12.112887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 19:45:13.115798: W tensorflow/compiler/tf

In [2]:
model = load_model("abalone_model.keras")

2024-11-21 19:45:23.146423: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


'{"predict_result": [[0.0]]}'

In [4]:
!pip install flask

In [5]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.1',image_scope='inference',instance_type='ml.c5.4xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu'

In [6]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.1-cpu-pluto"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 891377019371 ######################
region: us-east-1 ######################
fullname: 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-pluto ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-inference' does not exist in the registry with id '891377019371'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:891377019371:repository/tensorflow-inference",
        "registryId": "891377019371",
        "repositoryName": "tensorflow-inference",
        "repositoryUri": "891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
        "createdAt": 1732218482.024,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [10]:
!docker build -t 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-pluto-2 -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 357B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.2s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (10/10) FINISHED                               docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 357B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.2s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [11]:
!docker images

REPOSITORY                                                          TAG                  IMAGE ID       CREATED          SIZE
891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   2.12.1-cpu-pluto-2   6a69fe0aca43   20 minutes ago   4.53GB


In [12]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [13]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-pluto-2

The push refers to repository [891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

bb130fe6: Preparing 
cd1c4fb7: Preparing 
38463cbb: Preparing 
37bf1d61: Preparing 
8df97c44: Preparing 
764747b4: Preparing 
1a68f579: Preparing 
2eef5eea: Preparing 
b5bcc575: Preparing 
5e5ce62f: Preparing 
757a305d: Preparing 
c7cf6f28: Preparing 
30092134: Preparing 
fb153852: Preparing 
0903db8c: Preparing 
19dec72a: Preparing 
df04f233: Preparing 
f2dbc490: Preparing 
ba0431f9: Preparing 
71536788: Preparing 
994107ae: Preparing 
3a4f83e7: Preparing 
d6748243: Preparing 
f2c1e372: Preparing 
cd2b5d6d: Preparing 
d1c4fb7: Pushed   2.641GB/2.621GB6APushing  586.2MB/2.621GBPushing  699.2MB/2.621GBPushing    570MB/648.4MBPushing  1.301GB/2.621GBPushing  1.868GB/2.621GB2.12.1-cpu-pluto-2: digest: sha256:072f2ae0b25b2812bc9e6f7c3cf6113c4c1b9779e672a618d5638816703ef5a7 size: 5765


In [14]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'training-2023-11-23-18-45-31-074'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-pluto-2',
        'ModelDataUrl': 's3://asdnajsndalkas/data/output/training_job-2024-11-05-14-05-20/output/model.tar.gz',
    })

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://asdnajsndalkas/data/output/training_job-2024-11-05-14-05-20/output/model.tar.gz.

In [67]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'test-api-inference'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'test-endpoint-inference'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

Created EndpointConfig: arn:aws:sagemaker:us-east-1:891377019371:endpoint-config/test-api-inference
